In [ ]:
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Kkma
from konlpy.utils import pprint

import threading
import time

import time
import datetime
from datetime import timedelta
import schedule

import pymysql
from selenium.webdriver.chrome.options import Options

In [ ]:
def start_crawling_fun():
    start = time.time()
    
    # 페이지 접근
    url ='https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=102&sid2=249' # 사건사고
    driver.get(url)

    # 코드 추출
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    classToIgnore = ["photo"]
    news_write_time=[]

    # 10분 내에 작성된 기사 크롤링
    for n_t in range(0,20):
        news_time = int(re.sub("분전" , '', soup.find_all('span',{"class":"date is_new"})[n_t].get_text()))

        if news_time <= 10:
            news_title = [re.sub("\n|\t|[']" , '', soup.find_all('dt', class_=lambda x: x not in classToIgnore)[n].get_text()).replace('"','') for n in range(0,n_t+1)]
            news_href = [soup.find_all('dt', class_=lambda x: x not in classToIgnore)[n].find("a")["href"] for n in range(0,n_t+1)]
            news_writer = [soup.find_all('span', {"class":"writing"})[n].get_text() for n in range(0,n_t+1)]
            news_content = [re.sub("\n|\t|서울경제|[']" , '', soup.find_all('span',{"class":"lede"})[n].get_text()).replace('"','') for n in range(0,n_t+1)]
            news_write_time.append(datetime.datetime.now() - timedelta(minutes = news_time))
        else:
            break

    # 지역저장할 리스트        
    news_area = []

    # 지역저장
    for i in range(0,len(news_content)):
        news_area.append('NaN')
        stop = True
        for x in area:
            if (news_content[i].find(x) != -1)|(news_title[i].find(x) != -1):
                news_area[i] = x
                stop = False
                break    
        if stop == False:
            continue
        news_url = news_href[i]
        driver.get(news_url)
        news_html = driver.page_source
        news_soup = BeautifulSoup(news_html,'html.parser')
        for x in area:
            if news_soup.find('div',{"id":"articleBodyContents"}).get_text().find(x) != -1:
                news_area[i] = x
                break
        

    # 기사 저장할 데이터 프레임 생성
    news=pd.DataFrame({"title":news_title,"content":news_content,"href":news_href,"writer":news_writer,"write_time":news_write_time,"news_area":news_area})
    
    # 지역이 있는 뉴스만
    news = news[~ (news.news_area=='NaN')].reset_index()
    
    # 형태소 분석
    kkma = Kkma()

    title_kkma=[]

    for x in news.title:
        kkma_nouns = ' '.join(kkma.nouns(x))
        title_kkma.append(kkma_nouns)

    tfidf_vectorizer =  TfidfVectorizer(min_df=1)
    tfidf_matrix_kkma =  tfidf_vectorizer.fit_transform(title_kkma)

    document_kkma = (tfidf_matrix_kkma * tfidf_matrix_kkma.T)

    df = pd.DataFrame(document_kkma.todense())

    # 유사도가 50%이상인 기사 추출 및 제거
    for i in range(0,df.shape[0]):
        j = i+1
        while j < df.shape[1]:
            if df.iloc[i,j] >= 0.5:
                df = df.drop(df.index[j],axis=1)
                df = df.drop(df.index[j]) 
                j -= 1
            j += 1

    # DB 저장      

    for x in range(len(news.loc[list(df.columns)])):
        sql = """ INSERT INTO cctv_news(News_title,News_content,News_href,News_writer,News_write_time,News_area) VALUES('"""+str(list(news.loc[list(df.columns)].title)[x])+"""',
        '"""+str(list(news.loc[list(df.columns)].content)[x])+"""',
        '"""+str(list(news.loc[list(df.columns)].href)[x])+"""',
        '"""+str(list(news.loc[list(df.columns)].writer)[x])+"""',
        '"""+list(news.loc[list(df.columns)].write_time)[x].strftime("%Y-%m-%d %H:%M:%S")+"""',
        '"""+str(list(news.loc[list(df.columns)].news_area)[x])+"""');    
        """
        news_db.execute(sql)
        
    db.commit()
    
    print("time :", time.time() - start)

In [ ]:
if __name__ == '__main__':
    
    # db연결
    db = pymysql.connect(host='database-big-a.cljkqcsbb9ok.ap-northeast-2.rds.amazonaws.com',port=3306, user='edu',passwd='multi1234!', db='iseeyou',charset='utf8')
    news_db = db.cursor()
    news_db

# 지역구 이름 리스트
    area_all = ['용산구', '해운대구', '안산', '덕양구', '부평구', '강동구', '달서구', '분당구', '노원구', '성동구', '상당구', '대덕구', '양천구', '사상구', '영통구', '포항', '서대문구', '만안구', '연수구', '서초구', '마포구', '강남구', '마산회원구', '계양구', '동구', '금천구', '동래구', '고양', '처인구', '부산진구', '수지구', '광산구', '수성구', '팔달구', '인천', '은평구', '수원', '흥덕구', '의창구', '동남구', '단원구', '중원구', '안양', '진해구', '동작구', '유성구', '청주', '송파구', '영등포구', '영도구', '강서구', '울산', '장안구', '강북구', '대구', '일산동구', '성북구', '덕진구', '기흥구', '완산구', '부산', '남구', '북구', '동대문구', '서구', '광주', '대전', '서울', '미추홀구', '권선구', '금정구', '상록구', '중구', '관악구', '전주', '도봉구', '동안구', '종로구', '수영구', '서원구', '수정구', '연제구', '성산구', '사하구', '창원', '서북구', '구로구', '청원구', '남양주', '천안', '광진구', '마산합포구', '중랑구', '일산서구', '성남', '남동구', '용인']
    area = ['중구', '종로구', '동대문구', '성동구', '서대문구', '용산구', '영등포구', '마포구', '성북구', '중구', '서구', '영도구', '동구', '부산진구', '동래구', '중구', '동구', '서구', '남구', '북구', '중구', '동구', '남구', '미추홀구', '북구', '부평구', '도봉구', '관악구', '동구', '서구', '강남구', '남구', '강서구', '동구', '중구', '북구', '은평구', '강동구', '구로구', '동작구', '해운대구', '수성구', '북구', '사하구', '중구', '남구', '광산구', '송파구', '중랑구', '노원구', '서초구', '양천구', '금정구', '달서구', '서구', '남동구', '서구', '동구', '권선구', '장안구', '강서구', '유성구', '대덕구', '수정구', '중원구', '덕진구', '완산구', '분당구', '만안구', '동안구', '팔달구', '상당구', '흥덕구', '북구', '남구', '광진구', '강북구', '금천구', '연제구', '수영구', '사상구', '연수구', '계양구', '남구', '덕양구', '일산동구', '북구', '상록구', '단원구', '영통구', '일산서구', '처인구', '기흥구', '수지구', '동남구', '서북구', '의창구', '성산구', '마산합포구', '마산회원구', '진해구', '청원구', '서원구', '남양주']

# webdriver 객체 생성
    options = Options()
    options.add_argument("--headless")
    options.add_argument("window-size=1400,1500")
    driver= webdriver.Chrome(options=options)
    
    crawling = True

# 10분마다 크롤링
    schedule.every(10).minutes.do(start_crawling_fun)


    while crawling:
        schedule.run_pending()
        time.sleep(1)
    